Exploratory Data Anlysis for Presidential Elections by State and County

In [18]:
# load packges

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gp
from geopandas import GeoDataFrame
import fiona 
import descartes
from descartes import PolygonPatch

In [6]:
cd Resources

C:\Users\lizba\Desktop\Pres-Election-Partisan-Breakdown\Resources


# Presadential Elections by State and County 1976-2016

In [7]:
# load file that segments by county
file2 = "countypres_2000-2016.csv"
df = pd.read_csv(file2)
df.head()


,year,state,state_po,county,FIPS,office,candidate,party,candidatevotes,totalvotes,version
0,2000,Alabama,AL,Autauga,1001.0,President,Al Gore,democrat,4942.0,17208,20191203
1,2000,Alabama,AL,Autauga,1001.0,President,George W. Bush,republican,11993.0,17208,20191203
2,2000,Alabama,AL,Autauga,1001.0,President,Ralph Nader,green,160.0,17208,20191203
3,2000,Alabama,AL,Autauga,1001.0,President,Other,NaN,113.0,17208,20191203
4,2000,Alabama,AL,Baldwin,1003.0,President,Al Gore,democrat,13997.0,56480,20191203


perforem some groupby operations to learn more about dataset

In [14]:
# # votes per canidate per county
df.groupby(['state','county','candidate']).sum()
#TODO create a function for this groupby.coun

year      FIPS  candidatevotes  totalvotes  \
state   county  candidate                                                      
Alabama Autauga Al Gore           2000    1001.0          4942.0       17208   
                Barack Obama      4020    2002.0         12456.0       47573   
                Donald Trump      2016    1001.0         18172.0       24973   
                George W. Bush    4004    2002.0         27189.0       37289   
                Hillary Clinton   2016    1001.0          5936.0       24973   
...                                ...       ...             ...         ...   
Wyoming Weston  John Kerry        2004   56045.0           580.0        3392   
                John McCain       2008   56045.0          2618.0        3414   
                Mitt Romney       2012   56045.0          2821.0        3359   
                Other            10040  280225.0           585.0       16751   
                Ralph Nader       2000   56045.0            26.0        3060   

                                   version  
state   county  candidate                   
Alabama Autauga Al Gore           20191203  
                Barack Obama      40382406  
                Donald Trump      20191203  
                George W. Bush    40382406  
                Hillary Clinton   20191203  
...                                    ...  
Wyoming Weston  John Kerry        20191203  
                John McCain       20191203  
                Mitt Romney       20191203  
                Other            100956015  
                Ralph Nader       20191203  

[31528 rows x 5 columns]

In [26]:
# find out how many counties are in each state
county_counts = df.groupby(["county"]).count()

,year,state,state_po,FIPS,office,candidate,party,candidatevotes,totalvotes,version
county,,,,,,,,,,
Abbeville,16,16,16,16,16,16,11,16,16,16
Acadia,16,16,16,16,16,16,11,16,16,16
Accomack,16,16,16,16,16,16,11,16,16,16
Ada,16,16,16,16,16,16,11,16,16,16
Adair,64,64,64,64,64,64,44,62,64,64
...,...,...,...,...,...,...,...,...,...,...
Yuba,16,16,16,16,16,16,11,16,16,16
Yuma,32,32,32,32,32,32,22,32,32,32
Zapata,16,16,16,16,16,16,11,16,16,16


 add column for percentage

In [12]:
# add new columns for percent of total votes per county
df['percent'] = df['candidatevotes'] / df['totalvotes']
df.head()

,year,state,state_po,county,FIPS,office,candidate,party,candidatevotes,totalvotes,version,percent
0,2000,Alabama,AL,Autauga,1001.0,President,Al Gore,democrat,4942.0,17208,20191203,0.287192
1,2000,Alabama,AL,Autauga,1001.0,President,George W. Bush,republican,11993.0,17208,20191203,0.696943
2,2000,Alabama,AL,Autauga,1001.0,President,Ralph Nader,green,160.0,17208,20191203,0.009298
3,2000,Alabama,AL,Autauga,1001.0,President,Other,NaN,113.0,17208,20191203,0.006567
4,2000,Alabama,AL,Baldwin,1003.0,President,Al Gore,democrat,13997.0,56480,20191203,0.247822


just look at year 2016

In [13]:
# create boolean condition 
is_2016 = (df['year']==2016)
is_2016

0        False
1        False
2        False
3        False
4        False
         ...  
50519     True
50520     True
50521     True
50522     True
50523     True
Name: year, Length: 50524, dtype: bool

In [14]:
# Plug in True Boolean to new DF
df_2016 = df[is_2016]
df_2016.head()

,year,state,state_po,county,FIPS,office,candidate,party,candidatevotes,totalvotes,version,percent
40517,2016,Alabama,AL,Autauga,1001.0,President,Hillary Clinton,democrat,5936.0,24973,20191203,0.237697
40518,2016,Alabama,AL,Autauga,1001.0,President,Donald Trump,republican,18172.0,24973,20191203,0.727666
40519,2016,Alabama,AL,Autauga,1001.0,President,Other,NaN,865.0,24973,20191203,0.034637
40520,2016,Alabama,AL,Baldwin,1003.0,President,Hillary Clinton,democrat,18458.0,95215,20191203,0.193856
40521,2016,Alabama,AL,Baldwin,1003.0,President,Donald Trump,republican,72883.0,95215,20191203,0.765457


In [15]:
# rename column to match name in geopandas df
df_2016 = df_2016.rename(columns={'county':'COUNTYNAME'})


In [16]:
df_2016.head()

,year,state,state_po,COUNTYNAME,FIPS,office,candidate,party,candidatevotes,totalvotes,version,percent
40517,2016,Alabama,AL,Autauga,1001.0,President,Hillary Clinton,democrat,5936.0,24973,20191203,0.237697
40518,2016,Alabama,AL,Autauga,1001.0,President,Donald Trump,republican,18172.0,24973,20191203,0.727666
40519,2016,Alabama,AL,Autauga,1001.0,President,Other,NaN,865.0,24973,20191203,0.034637
40520,2016,Alabama,AL,Baldwin,1003.0,President,Hillary Clinton,democrat,18458.0,95215,20191203,0.193856
40521,2016,Alabama,AL,Baldwin,1003.0,President,Donald Trump,republican,72883.0,95215,20191203,0.765457


# Getting Shape file to load into GeoPandas

In [38]:
# load shp file extracted from zipfile
county_shp = gp.read_file("c_03mr20.shp" )
county_shp.head(2)

,STATE,CWA,COUNTYNAME,FIPS,TIME_ZONE,FE_AREA,LON,LAT,geometry
0,ME,CAR,Washington,23029,E,se,-67.6361,45.0363,"MULTIPOLYGON (((-67.93539 44.40382, -67.93643 ..."
1,GA,CHS,McIntosh,13191,E,se,-81.2646,31.5329,"MULTIPOLYGON (((-81.46814 31.33980, -81.46747 ..."


In [40]:
county_shp['FIPS'] = county_shp['FIPS'].astype('float')
county_shp


,STATE,CWA,COUNTYNAME,FIPS,TIME_ZONE,FE_AREA,LON,LAT,geometry
0,ME,CAR,Washington,23029.0,E,se,-67.6361,45.0363,"MULTIPOLYGON (((-67.93539 44.40382, -67.93643 ..."
1,GA,CHS,McIntosh,13191.0,E,se,-81.2646,31.5329,"MULTIPOLYGON (((-81.46814 31.33980, -81.46747 ..."
2,GA,CHS,Liberty,13179.0,E,se,-81.2103,31.7093,"POLYGON ((-81.30807 31.79454, -81.30546 31.791..."
3,AS,PPG,Swains Island,60040.0,S,None,-171.0459,-11.0843,"POLYGON ((-171.04049 -11.08245, -171.03940 -11..."
4,AS,PPG,Manu'a,60020.0,S,None,-169.5060,-14.2219,"MULTIPOLYGON (((-169.61275 -14.15533, -169.611..."
...,...,...,...,...,...,...,...,...,...
3326,VA,AKQ,York,51199.0,E,se,-76.5637,37.2371,"POLYGON ((-76.71140 37.36571, -76.70710 37.365..."
3327,TX,BRO,Kenedy,48261.0,C,ds,-97.7174,26.9281,"MULTIPOLYGON (((-97.31581 26.60280, -97.31620 ..."
3328,TX,BRO,Willacy,48489.0,C,ds,-97.6732,26.4706,"MULTIPOLYGON (((-97.44247 26.37530, -97.44263 ..."
3329,MI,APX,Chippewa,26033.0,E,ee,-84.5921,46.3031,"MULTIPOLYGON (((-83.56822 45.92122, -83.56780 ..."


In [41]:
# join the geodataframe with the cleaned up csv 
#merged = df2.set_index('state').join(county_shp.set_index('STATE'))

merged = df_2016.merge(county_shp, how='outer')

In [42]:
try_this = county_shp.merge(df_2016, on='FIPS')
try_this.sample(5)

,STATE,CWA,COUNTYNAME_x,FIPS,TIME_ZONE,FE_AREA,LON,LAT,geometry,year,state,state_po,COUNTYNAME_y,office,candidate,party,candidatevotes,totalvotes,version,percent
233,PA,CTP,Perry,42099.0,E,sc,-77.2622,40.3984,"POLYGON ((-76.95319 40.60291, -76.95440 40.600...",2016,Pennsylvania,PA,Perry,President,Other,NaN,910.0,21158,20191203,0.043010
8255,WY,RIW,Lincoln,56023.0,M,wc,-110.6556,42.2624,"POLYGON ((-110.92790 43.31591, -110.92780 43.3...",2016,Wyoming,WY,Lincoln,President,Other,NaN,991.0,8875,20191203,0.111662
7801,TX,FWD,Wise,48497.0,C,nc,-97.6544,33.2159,"POLYGON ((-97.48740 33.43381, -97.46700 33.433...",2016,Texas,TX,Wise,President,Donald Trump,republican,20670.0,24776,20191203,0.834275
7836,TX,LUB,Castro,48069.0,C,pa,-102.2617,34.5300,"POLYGON ((-101.99930 34.74821, -101.99850 34.7...",2016,Texas,TX,Castro,President,Hillary Clinton,democrat,526.0,1997,20191203,0.263395
7685,TX,SJT,San Saba,48411.0,C,wc,-98.8176,31.1552,"POLYGON ((-99.01880 31.47021, -99.01610 31.470...",2016,Texas,TX,San Saba,President,Other,NaN,39.0,2357,20191203,0.016546


In [ ]:
try_this['FIPS'].astype()

In [31]:
type(try_this)

geopandas.geodataframe.GeoDataFrame